# Decision Optimization Notebook - Retail SC Optimization 

This is an example of a 'traditional' notebook approach to implement an optimization model.<br>
You can run this with a Default Python 3.6 environment in case you run into challenges with Default Python 3.7 environment.

Stand-alone/traditional in the sense:
* This notebook can be used as a solver notebook in DO4WS.
* Uses a few stand-alone re-usable functions defined at the start of the notebook. Includes only those functions it needs.
* All inputs, outputs and intermediate data DataFrames are defined globally.
* No object-oriented code.
* No re-use and sharing of code (no import from custom modules or packages).
* No use of `dse-do-utils`

## Table of Contents:
* [References](#references)
* [Read Me](#Read_Me)
* [Package Management](#package_management)
* [Instantiate CPLEX](#instantiate_cplex)
* [Read Data Inputs](#read_data)
* [Create Decision Variables](#decision_variables)
* [Objective Function Definition](#objective)
* [Constraint Definition](#constraint)
* [Run CPLEX Model](#run_model)
* [Post Processing](#post)

## References<a class="anchor" id="references"></a>
* Authors | Jesse Slater & Wesley Beaver | IBM Cloud Pak Acceleration Team
* Contributor | Victor Terpstra | IBM Senior Data Scientist DSE | https://github.ibm.com/vterpstra/DO_Template_Projects_CPD25
* GitHub CPLEX Documentation | http://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html

# READ ME<a class="anchor" id="Read_Me"></a>

This notebook solves a simplified production planning model that is part of a larger supply chain optimization problem.

The supply-chain model we'll be building is a simplified version that only considers production facilities and warehouses (Suppliers, Manufacturers, Warehouse) and demand, i.e.:
* Multiple plants, from multiple suppliers in multiple locations.
* Same line may be capable of producing a range of products, each which their own production rate and cost
* Multiple products, where different suppliers compete, i.e. are capable of producing the same product.
* Multiple warehouses and different warehouse locations
* Warehouses have demand for the various products at various locations

Simplification:
* PLEASE NOTE: In order to set a flexible foundation for various client environments, simplification must be made in lieu of company specific data. Any of the following simplifications can be resolved with actual or estimated historical cost/demand/route data.
* Scenario data including routes, estimated costs, current inventory has been affixed to the LSTM forecasted demand and saved to All_Routes_Table.csv
* Transportation cost is a hardcoded linear cost pending straight line point to point travel distance (Google API can be substituted to find route distance by roadmap)
* Transportation time is the summation of number of stops after the supplier. A route passing from one supplier to one manufacturer to one warehouse will be 2 days
* Demand forecast has already been generated by LSTM model (or other user defined model) and saved to data assets
* Optimization model will solve for the first demand period generated by the forecast
* Each supplier/manufacturer combination can produce product in demand
* No inventory
* User has enabled the decision optimization cartridge on Cloud Pak for Data 3.0+

Supply Chain optimization will plan:
* Production activities: how much of which product to produce on which line for which warehouse.
* Backlog: Whatever demand cannot be met will be stored into an order backlog

Such that:
* Minimal production cost
* Minimal travel cost
* Minimal backlog cost
* Meet demand
* Below or equal to supplier capacity

The input tables for this model:
* Product
* Location
* Plant
* Line
* Demand

# Package Management<a class="anchor" id="package_management"></a>
#### Use the following code block to install any missing packages

In [1]:
!pip install dd_scenario -q

In [2]:
!pip install docplex -q

#### Import packages and print environment health check

In [3]:
import pandas as pd
from IPython import get_ipython
import os
from tqdm.auto import tqdm
import numpy as np
import dd_scenario as dd
from docplex.mp.environment import Environment
from docplex.mp.model import Model
import warnings
warnings.filterwarnings('ignore')
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.6.10, located at: /opt/conda/envs/Python-3.6-WMLCE/bin/python
* docplex is present, version is (2, 12, 182)
* CPLEX library is present, version is 12.10.0.0, located at: /user-home/_global_/python-3.6
* pandas is present, version is 0.24.2


#### Prepare .lp and .cpo exports to save model later on

In [4]:
import os

def WSL_export_as_lp(model, model_name = None, local_dir = None, WSL_dir = 'datasets'):
    '''Export .lp file of model in the 'WSL_PROJECT_DIR.datasets' folder as a .csv file, so it can be exported to a local machine.
    If not in WSL, it will write to the local file system in the 'local_dir' directory.
    If neither in WSL or the local_dir is not defined, no .lp file is exported.
    
    Args:
        model (docplex.mp.model): The CPLEX model to be exported
        model_name (str): name of .lp file. If none specified, will use the model.name
        local_dir (str): name of local directory. Will write .lp file here, if specified and if not in WSL
        WSL_dir (str): name of directory in WSL_PROJECT_DIR. Default = 'datasets'
    Returns: 
        path (str) path to lp file
    '''
    
    if model_name is None:
        model_name = model.name
        
    if local_dir is not None:
        lp_file_name = os.path.join(local_dir, model_name+'.lp')
        model.export_as_lp(lp_file_name)
        return lp_file_name
    else:
        return None
    
def WSL_export_as_cpo(model, model_name = None, local_dir = None, WSL_dir = 'datasets', **kwargs):
    '''Export .cpo file of model in the 'WSL_PROJECT_DIR.datasets' folder as a .csv file, so it can be exported to a local machine.
    If not in WSL, it will write to the local file system in the 'local_dir' directory.
    If neither in WSL or the local_dir is not defined, no .cpo file is exported.
    
    Args:
        model (docplex.cp.model): The CPO to be exported
        model_name (str): name of .cpo file. If none specified, will use the model.name
        local_dir (str): name of local directory. Will write .cpo file here, if specified and if not in WSL
        WSL_dir (str): name of directory in WSL_PROJECT_DIR. Default = 'datasets'
    Returns:
        path (str): path to cpo file
    '''
    
    if model_name is None:
        model_name = model.name

    if local_dir is not None:
        lp_file_name = os.path.join(local_dir, model_name+'.lp')
        model.export_as_lp(lp_file_name)
        return lp_file_name
    
    else:
        return None

# Instantiate CPLEX<a class="anchor" id="instantiate_cplex"></a>
#### Set CPLEX characteristics

In [5]:
DO_MODEL_NAME = 'SC_DO'
SCENARIO_NAME = 'Warehouse_Demand'#'Scenario 1'
TIME_LIMIT = 30

#### Retrieve access token and instantiate an optimization model we will call shortly

In [6]:
access_token = os.environ['USER_ACCESS_TOKEN']
model_builder = dd.ModelBuilder(name=DO_MODEL_NAME, projectId="pp-id")

# Creates a client...
client = dd.Client(project_id="pp-id",
                authorization=access_token)
mdl = Model(name="ProductionPlanning")

# Read Data Inputs<a class="anchor" id="read_data"></a>

#### Pull in the demand and forecast data
The first csv file contains every possible route (534,096) that a given product can take from supplier, manufacturer to warehouse. Note, that there are 4 suppliers, 3 manufacturers and 4 warehouses. Additionally, this file contains the travel costs (as described in distance of the route) and production capacity for each supplier product combination. 

The second csv file contains the forecasts for the next three months which were output by the neural network in the notebook `1-Modeling`.

In [7]:
# Demand
SC_data = pd.read_csv('/project_data/data_asset/All_Routes_Table.csv')
SC_data['Product_Code_Warehouse'] = SC_data['Product_Code'] + '_' + SC_data['Warehouse']
SC_data.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2             5   
1        W2  Product_0001         S1      M1  W1  W4  W3  W2             5   
2        W2  Product_0001         S1      M1  W3  W1  W4  W2             5   
3        W2  Product_0001         S1      M1  W3  W4  W1  W2             5   
4        W2  Product_0001         S1      M1  W4  W1  W3  W2             5   

  Destination  ...           f3           f4          f5  capacity  \
0          W2  ...  1941.680672  1574.236047  965.635097      58.0   
1          W2  ...  1238.977201   987.316035  378.715085      58.0   
2          W2  ...  1584.740888  1217.296263  965.635097      58.0   
3          W2  ...  1599.191579   990.590629  738.929463      58.0   
4          W2  ...   997.820877   746.159710  378.715085      58.0   

                      prodx  cost  Product_Code_Warehouse Forecast  \
0  Product_0001S1M1W1W3W4W2  0.02         Product_0001_W2      138   
1  Product_0001S1M1W1W4W3W2  0.02         Product_0001_W2      138   
2  Product_0001S1M1W3W1W4W2  0.02         Product_0001_W2      138   
3  Product_0001S1M1W3W4W1W2  0.02         Product_0001_W2      138   
4  Product_0001S1M1W4W1W3W2  0.02         Product_0001_W2      138   

                 xProd_vecm production_quantity  
0  Product_0001S1M1W1W3W4W2                 0.0  
1  Product_0001S1M1W1W4W3W2                 0.0  
2  Product_0001S1M1W3W1W4W2                 0.0  
3  Product_0001S1M1W3W4W1W2                 0.0  
4  Product_0001S1M1W4W1W3W2                 0.0  

[5 rows x 23 columns]

In [8]:
forecasts = pd.read_csv('/project_data/data_asset/Next_Three_Month_Forecasts.csv')
forecasts = forecasts.rename(columns={'Unnamed: 0': 'Date'})
forecasts.head()

Date  Product_0001_W2  Product_0003_W2  Product_0004_W2  \
0  2017-01-01             1726             1772             2305   
1  2017-02-01             2113              971             1648   
2  2017-03-01              131             2814             2374   

   Product_0005_W2  Product_0006_W2  Product_0007_W2  Product_0008_W2  \
0                0                2             6815             6344   
1               57              394             6217            19657   
2             1156              604             5069             9579   

   Product_0009_W2  Product_0010_W2  ...  Product_2138_W1  Product_2142_W1  \
0             7785             1604  ...               48                0   
1             3010              846  ...              428                8   
2             4651             1737  ...              239                0   

   Product_2144_W1  Product_2147_W1  Product_2164_W1  Product_2165_W1  \
0               71                3                1                5   
1              381                3                6                4   
2              198                2               13               17   

   Product_2166_W1  Product_2167_W1  Product_2168_W1  Product_2169_W1  
0               22              135                0               13  
1               33              175               11                8  
2               11              292               29               19  

[3 rows x 2850 columns]

#### Pull in the demand data
This dataset provides the forecast demand for every warehouse product combination available. 

In [9]:
# Filter SC data to keep only combinations that exist in forecasts and get the forecasts

for col_name in tqdm(forecasts.columns[1:]):
    if col_name == forecasts.columns[1]:
        main_df = SC_data[SC_data['Product_Code_Warehouse'] == col_name]
        main_df['Forecast'] = forecasts[col_name][len(forecasts[col_name]) - 1]
    else:
        df = SC_data[SC_data['Product_Code_Warehouse'] == col_name]
        df['Forecast'] = forecasts[col_name][len(forecasts[col_name]) - 1]
        main_df = pd.concat([main_df, df], axis=0)

In [10]:
demand_df = main_df[['Destination','Product_Code','Forecast']].drop_duplicates().reset_index(drop = True)
demand_df.head()

Destination  Product_Code  Forecast
0          W2  Product_0001       131
1          W2  Product_0003      2814
2          W2  Product_0004      2374
3          W2  Product_0005      1156
4          W2  Product_0006       604

#### Pull in our backlog data (currently inventory)
This csv file contains the backlog for every product in every warehouse.

In [11]:
# Backlog
backlog_df = pd.read_csv('/project_data/data_asset/Backlog.csv')
backlog_demand = backlog_df.merge(demand_df, left_on = ('Destination', 'Product_Code'), right_on= ('Destination', 'Product_Code'))
backlog_demand.head()

Destination  Product_Code  backlog  Forecast
0          W2  Product_0001      7.0       131
1          W2  Product_0003    229.0      2814
2          W2  Product_0004     93.0      2374
3          W2  Product_0005     63.0      1156
4          W2  Product_0006     29.0       604

# Create decision variables<a class="anchor" id="decision_variables"></a>
#### Re-usable functions to emphasize best-practice pattern
#### Added as new method on a CPLEX Model so usage is very close to a call to the other methods of Model

In [12]:
from docplex.mp.model import Model

def integer_var_series(self, df, **kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl (docplex.mp.model): CPLEX Model
        df (DataFrame): dataframe
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
    Returns:
        (pandas.Series) with integer dvars, index matches index of df
    '''
    # We are re-using the index from the DF index:
    return pd.Series(self.integer_var_list(df.index, **kargs), index = df.index)

def continuous_var_series(self, df, **kargs):
    return pd.Series(self.continuous_var_list(df.index, **kargs), index = df.index)

def binary_var_series(self, df, **kargs):
    return pd.Series(self.binary_var_list(df.index, **kargs), index = df.index)

# Bind method to class:
Model.integer_var_series = integer_var_series
Model.continuous_var_series = continuous_var_series
Model.binary_var_series = binary_var_series

### Data Checker<a class="anchor" id="data_checker"></a>
For now assume that input data originates from a relational database that ensures:
* All the required columns are present
* The names of the columns are as expected
* Each column has the expected data type
* Null vs None-null values as expected
* Relational integrity: foreign keys exist as rows in their source table

### Pre-processing<a class="anchor" id="pre_procesing"></a>
The basic building blocks for this optimization problem are the `ProductionActivities` and `AggregateDemand`, i.e. what Product to produce on what Line:
* (key) Product
* (key) Line
* quantity

Steps:
1. Get active Demand and active Products
2. Create ProductionActivities that can manufacture a Product on a Production Line in a TimePeriod
3. Aggregate Demand

### ProductionActivities 
1. Create all combinations between a Product and Product Line.
2. Create all combinations between a Product, Warehouse and Production Line.

Production Activities are the basis for the production quantity decision variables.

### AggregateDemand
AggregateDemand is the basis for the demand fulfillment constraints and the backlog decision variables.<br>
We aggregate demand over each product, so we can define a backlog per product.<br>
In this simplified demo, we do not yet model backlog per time-period, which requires a backlog-resupply.

# Objective Function Definition<a class="anchor" id="objective"></a>

In [13]:
from IPython.display import clear_output

# Create production activities and aggregate demand tables for clarity
pa = main_df
ad = backlog_demand
backlog_penalty = 10 # For now, hard-code the back log penalty
pa.Miles = pa.Miles/100000 # For now, hard-code cost. Drive down milage to create simplified cost per product

name_f = tuple(list(pa.prodx))
pa['xProd_vecm'] = mdl.integer_var_series(pa, lb = 0, name = name_f)
clear_output()

#### Minimize cost:
-  Production cost:
    -  _sum(pa) xProd[pa] * pa.costPerPart_
    -  i.e. the production quantity * the production cost per part
-  Backlog cost: 
    -  _sum(ad) xBacklog[ad] * backlogPenalty_
    -  i.e. the backlog quantity * backlog penalty. For now the backlog penalty is a static number (100)
    
The backlog costs are set an order of magnitude higher than the production cost. This will force the model to make backlog reduction the highest priority, before minimizing the production cost.<br>
_Note: an alternative approach is to define a lexicographical/hierarchical optimization approach._

#### Production Cost: 
The production cost is the amount of units supplied per given product route multiplied by the production cost.

$$
productionCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * pa.costPerPart
$$

In [14]:
production_cost = mdl.sum(pa.xProd_vecm * pa.cost) #Uses vector multiplication, very efficient
mdl.add_kpi(production_cost, "Production Cost")

DecisionKPI(name=Production Cost,expr=0.020Product_0001S1M1W1W3W4W2+0.020Product_0001S1M1W1W4W3W2+0.02..)

#### Backlog Cost: 
Backlog cost is the number of products that can't immediately be created due to production capacity constraints for each warehouse multiplied by the penalty assigned for each backlog unit. 
$$
backlogCost = \sum\limits_{\forall AggregatedDemand} xBacklog[ad] * penalty
$$

In [15]:
ad['xBacklog'] = ad['Destination'] + ad['Product_Code']
backlog_f = tuple(list(ad.xBacklog))
ad['xBacklog_vecm'] = mdl.integer_var_series(ad, lb = 0, name = backlog_f)

In [16]:
backlog_cost = mdl.sum(ad.xBacklog_vecm * backlog_penalty)
mdl.add_kpi(backlog_cost, "Backlog Cost")

DecisionKPI(name=Backlog Cost,expr=10W2Product_0001+10W2Product_0003+10W2Product_0004+10W2Product_0..)

#### Travel Cost: 
Travel cost is the number of units supplied for each unique product route multiplied by the distance (as measured in miles).
$$
TravelCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * xMile
$$

In [17]:
travel_cost = mdl.sum(pa.xProd_vecm * pa.Miles)
mdl.add_kpi(travel_cost, "Travel Cost")

DecisionKPI(name=Travel Cost,expr=0.000Product_0001S1M1W1W3W4W2+0.000Product_0001S1M1W1W4W3W2+0.00..)

#### Objective Function:
In this use case, our objective function attempts to minimize the backlog, production and travel costs. The cell below takes the various costs that we have created above and defines our objective function. 

In [18]:
mdl.minimize(backlog_cost + production_cost + travel_cost)

# Constraint Definition<a class="anchor" id="constraint"></a>
-  Production capacity:
    -  _For each line l:_ 
        -  _sum(pa in l) xProd[pa] / lineCapacity[l,pa.p] <= 1_
    -  Each line has a different capacity to produce a product. Therefore, we compute an expression for the line utilization per pa. The total utilization must be below 100%
-  Fulfill demand:
    -  _For each aggregated demand product p:_ 
        -  _sum(pa in p) xProd[pa] >= p.aggregatedDemand - xBacklog[p]_
    -  The total production for a part needs to be larger than the total demand, minus the backlog

In [19]:
ad.head()

Destination  Product_Code  backlog  Forecast        xBacklog   xBacklog_vecm
0          W2  Product_0001      7.0       131  W2Product_0001  W2Product_0001
1          W2  Product_0003    229.0      2814  W2Product_0003  W2Product_0003
2          W2  Product_0004     93.0      2374  W2Product_0004  W2Product_0004
3          W2  Product_0005     63.0      1156  W2Product_0005  W2Product_0005
4          W2  Product_0006     29.0       604  W2Product_0006  W2Product_0006

In [20]:
# create forecast compensating for backlog
ad['forecast_backlog'] = ad['Forecast'] - ad['backlog']

#### Demand Fulfillment Constraint: 
This constraint stipulates that for every product and its final warehouse destination the quantity supplied is always greater than or equal to the forecast minus the original backlog as well as any additional backlog added. 

In [21]:
# Demand Fulfillment Constraint
for (product, warehouse), group in pa.groupby(['Product_Code', 'Destination']):
    ad1 = ad[(ad['Product_Code']==product) & (ad['Destination']==warehouse)].sum()
    mdl.add_constraint(
        mdl.sum(group.xProd_vecm) 
         >= 
        ad1.forecast_backlog - ad1.xBacklog_vecm, 
         'DemandFulfillmentC_{}'.format(product)
    )

#### Capacity Constraint: 
This constraint stipulates that the amount of quantity supplied for every product supplier combination 
does not exceed the amount that the given supplier can create for a given product. 


In [22]:
# Capacity Constraint
for (product, supplier), group in pa.groupby(['Product_Code', 'SupplierID']):
    mdl.add_constraint(
        (mdl.sum(mdl.sum(group.xProd_vecm * (1/group.capacity)) <= 1)), 'WarehouseCapacityC_{}_{}'.format(product, supplier))

#### Define timeout: 
Optimization model does not endlessly try to find an optimal solution.

In [23]:
mdl.parameters.timelimit = TIME_LIMIT
print(mdl.parameters.print_info_to_string(print_defaults=False)) #Logs all parameter values that are different from the default

parameters.timelimit = 30.00000000000000



# Run CPLEX Model<a class="anchor" id="run_model"></a>
#### Now that our model is defined with an objective function and constraints, we can solve!

In [24]:
# CPLEX:
# Additionally you can add a time parameter to the solve method below: cplex_parameters = {'timelimit':TIME_LIMIT})
msol = mdl.solve(log_output=True)
if msol is not None:
    mdl.report()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               30
Tried aggregator 4 times.
MIP Presolve eliminated 7607 rows and 451666 columns.
MIP Presolve modified 3454 coefficients.
Aggregator did 397 substitutions.
Reduced MIP has 2111 rows, 6407 columns, and 8157 nonzeros.
Reduced MIP has 267 binaries, 6140 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.72 sec. (521.20 ticks)
Found incumbent of value 5.5532507e+08 after 0.77 sec. (552.34 ticks)
Probing time = 0.00 sec. (0.10 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 791 rows and 3979 columns.
Aggregator did 7 substitutions.
Reduced MIP has 1313 rows, 2421 columns, and 4156 nonzeros.
Reduced MIP has 101 binaries, 2320 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (32.00 ticks)
Probing time = 0.00 sec. (0.05 ticks)
Tried aggregator 1 time.
Detecti

#### Conflict resolution<a class="anchor" id="conflict_resolution"></a>
For larger sized decision models, it can be hard to find what is causing the model solve to fail. This is where the CPLEX conflict refiner can help. If the model is infeasible, the conflict refiner will search for the minimal conflict. The minimal conflict is the smallest set of constraints and variables that are in conflict with each other, i.e. where it is impossible to find a feasible solution among the decision variables that satisfy all constraints in the set.

Knowing the conflict can help understanding the modelling error and thereby help debugging the model generation code.

For details, see the reference manual: http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.conflict_refiner.html

Run the CPLEX conflict resolution to find out what constraints are conflicting:

In [25]:
from docplex.mp.conflict_refiner import ConflictRefiner
if msol is None:
    crefiner = ConflictRefiner() # Create an instance of the ConflictRefiner
    conflicts = crefiner.refine_conflict(mdl) # Run the conflict refiner
    for c in conflicts:
        print(c.element) # Display conflict result in a little more compact format than ConflictRefiner.display_conflicts

# Post Processing<a class="anchor" id="post"></a>

#### The cell below provides the total production, backlog, and travel costs - respectively

In [26]:
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
df_kpis

kpi         value
0  Production Cost  5.907225e+06
1     Backlog Cost  5.121938e+08
2      Travel Cost  8.472102e+00

#### Extract solution
-  ProductionActivity
-  Backlog

*Note: The reason we explicitly extract the solution in a separate column, versus indirectly referring to the value using the `.solution_value` attribute of the dvar is that the column value will allow us to do vector operations to compute derived values.*

In [27]:
pa['production_quantity'] = [dvar.solution_value for dvar in pa.xProd_vecm]
ad['backlog_quantity'] = [dvar.solution_value for dvar in ad.xBacklog_vecm]
pa_nozeros = pa[pa['production_quantity']>0]

product_category = pd.read_csv('/project_data/data_asset/Product_Category_Legend.csv')
product_category.columns = ['Index', 'Product_Category', 'Product_Code', 'Total']
del product_category['Index']
del product_category['Total']

location_df = pd.read_csv("/project_data/data_asset/Locations.csv")
location_df=location_df.rename(columns = {'Warehouse':'Warehouse_City'})

In [28]:
pa_withproduct = pa_nozeros.merge(product_category,how='left',left_on='Product_Code',right_on='Product_Code')
pa_prodandlocation = pa_withproduct.merge(location_df,how='left',left_on='Destination',right_on='WarehouseID')

#### Save the solution and output to our project

In [29]:
from project_lib import Project
project = Project.access()
project.save_data("Production_and_Location.csv", pa_prodandlocation.to_csv(index=False), overwrite = True)

{'file_name': 'Production_and_Location.csv',
 'message': 'File saved to project storage.',
 'asset_id': 'f1791140-19bd-4671-a7a0-b358c46836ce'}

#### Save the cplex model to our project

In [30]:
# Note: The local directory variable must be set before calling the function or the model will not save
path = WSL_export_as_lp(mdl, local_dir='/project_data/data_asset/')

if path is not None:
    print('Saved .lp file as ' + path)
else:
    print('No local_dir specified: cannot write .lp file')

Saved .lp file as /project_data/data_asset/ProductionPlanning.lp


**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**